In [1]:
import numpy as np
from scipy.optimize import minimize_scalar
from matplotlib import pyplot as plt

HW1

In [17]:
#Problem 1
def find_initial_guess_interval(f, start=0, step_size=0.5, max_steps=1000):
    """
    Find an initial guess interval [a, b] of length 1 for the Bisection Method.

    Parameters:
    - f: The function for which to find the root.
    - start: The starting point for the search.
    - step_size: The step size to use when moving from the start point.
    - max_steps: The maximum number of steps to take from the start point.

    Returns:
    - A tuple (a, b) representing the interval, or None if no change in sign is found.
    """
    x_prev = start
    f_prev = f(x_prev)
    for _ in range(max_steps):
        # Move to the next point
        x_next = x_prev + step_size
        f_next = f(x_next)
        
        # Check if we've found a change in sign
        if f_prev * f_next < 0:
            # Adjust to ensure the interval is of length 1
            a = np.floor(x_prev)
            b = a + 1
            return (a, b) if f(a) * f(b) < 0 else (b, b + 1)
        
        # Prepare for the next iteration
        x_prev, f_prev = x_next, f_next
        
        # Invert direction if reaching the end of the step limit
        if _ == max_steps // 2:
            step_size = -step_size
    
    print("Failed to find a change in sign. Try adjusting the start point or step size.")
    return None

def bisection_method(f, a, b, tol=1e-5, max_iterations=1000):
    if f(a) * f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    for n in range(1, max_iterations + 1):
        x_n = (a_n + b_n) / 2
        f_x_n = f(x_n)
        if f(a_n) * f_x_n < 0:
            b_n = x_n
        elif f(b_n) * f_x_n < 0:
            a_n = x_n
        elif f_x_n == 0:
            print("Found exact solution.")
            return x_n
        else:
            print("Bisection method fails.")
            return None
        if abs(b - a)/2**n <= tol:  #e_n <= epsilon; abs(b_n - a_n) <= tol can also work
            return x_n
    print("Exceeded maximum iterations.")
    return (a_n + b_n) / 2

def equation1(x): return np.exp(x)-3*x**2
def equation2(x): return x**3-x**2-x-1
def equation3(x): return np.exp(x)-(1/(0.1+x**2))
def equation4(x): return x-1-0.3*np.cos(x)

# Find initial guess intervals for each equation
interval1 = find_initial_guess_interval(equation1)
interval2 = find_initial_guess_interval(equation2)
interval3 = find_initial_guess_interval(equation3)
interval4 = find_initial_guess_interval(equation4)

# Apply the bisection method using the found intervals and print results
def process_equation(equation, interval, equation_number):
    if interval:
        root = bisection_method(equation, *interval)
        print(f"Equation {equation_number}: Interval found is {interval}. The root found is: {root}")
    else:
        print(f"Equation {equation_number}: No valid interval found.")

process_equation(equation1, interval1, 1)
process_equation(equation2, interval2, 2)
process_equation(equation3, interval3, 3)
process_equation(equation4, interval4, 4)

Equation 1: Interval found is (0.0, 1.0). The root found is: 0.9100112915039062
Equation 2: Interval found is (1.0, 2.0). The root found is: 1.8392868041992188
Equation 3: Interval found is (0.0, 1.0). The root found is: 0.6497573852539062
Equation 4: Interval found is (1.0, 2.0). The root found is: 1.1284255981445312


In [ ]:
def max_second_derivative(d2f, bounds):
    # Minimize the negative of the second derivative to find the maximum
    result = minimize_scalar(lambda x: -d2f(x), bounds=bounds, method='bounded')
    if result.success:
        return -result.fun  # Return the negative of the minimized value, which is the maximum
    else:
        raise ValueError("Could not find maximum of second derivative")

def newton_raphson_method_improved(f, df, d2f, x0, tol=1e-5, max_iterations=1000):
    xn = x0
    xn_previous = x0  # Initialize xn_previous for the first iteration comparison
    for n in range(1, max_iterations+1):  # Start loop from 1 for clearer iteration count
        fxn = f(xn)
        # Check if the function value at xn is 0
        if abs(fxn) == 0:
            return xn # Solution found, return xn
        
        dfxn = df(xn)
        if dfxn == 0:
            return "Zero derivative. No solution found."
        xn_next = xn - fxn / dfxn
        
        # Calculate maximum of the second derivative for the error bound between xn and xn_next
        try:
            max_d2f = max_second_derivative(d2f, (min(xn, xn_next), max(xn, xn_next)))
        except ValueError as e:
            return str(e), None

        error_bound = (xn_next - xn)**2 * max_d2f / (2 * abs(dfxn))
        if error_bound <= tol and n > 1:  # Ensure at least one iteration for comparison
            return xn  # Return xn because error bound condition is satisfied
        
        if n > 1 and not (abs(xn_next - xn) <= abs(xn - xn_previous)):
            return "Convergence not improving based on step size.", None

        xn_previous = xn
        xn = xn_next

    return "Exceeded maximum iterations or solution not found within tolerance.", None

# Note: Define f, df, d2f for your specific function before using this function.
